In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    f1_score,
    roc_auc_score,
    precision_recall_curve,
    auc,
    confusion_matrix,
    classification_report,
    accuracy_score,
    average_precision_score,
)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

2025-06-05 20:13:22.091776: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749168805.569283 3321552 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749168805.669080 3321552 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749168806.746410 3321552 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749168806.746469 3321552 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749168806.746475 3321552 computation_placer.cc:177] computation placer alr

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
print("GPUs visible to TF:", tf.config.list_physical_devices("GPU"))  

GPUs visible to TF: []


In [6]:
# === PATHS & SETTINGS ===
base_dir = '/.../.../.../.../Unsupervised_Evaluation_LLM/PaperWork'
folders = ['DNA_BERT2', 'HyenaDNA', 'mistral', 'nucleotide-transformer-500m-human-ref', 'genalm', 'grover']
dataset_names = ['demo_coding_vs_intergenomic_seqs', 'human_enhancers_cohn',
                 'human_enhancers_ensembl', 'human_nontata_promoters']
embedding_sources = ['DB2', 'HD', 'mistral', 'NT', 'GENALM', 'GROVER']

# === Load All Embedding Matrices ===
matrices = {}
for folder in folders:
    print(f"Loading from folder: {folder}")
    folder_path = os.path.join(base_dir, folder)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.npy'):
            file_path = os.path.join(folder_path, file_name)
            print(f"  Reading file: {file_name}")
            if folder == 'nucleotide-transformer-500m-human-ref':
                emb_file = np.load(file_path)
                matrices[file_name] = np.asarray(emb_file)
            else:
                emb_file = np.load(file_path, allow_pickle=True)
                matrices[file_name] = np.vstack(np.array(emb_file))

# === Helper Functions ===
def sensitivity_specificity(y_true, y_pred):
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    sensitivity = TP / (TP + FN + 1e-7)
    specificity = TN / (TN + FP + 1e-7)
    return sensitivity, specificity

def build_1d_cnn(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    return model

# === Run Pipeline ===
all_results = []

for dataset_name in dataset_names:
    print(f"\n\n=== Processing Dataset: {dataset_name} ===")
    
    # Load CSVs
    train_df = pd.read_csv(f'{base_dir}/datasets/train_{dataset_name}.csv')
    test_df  = pd.read_csv(f'{base_dir}/datasets/test_{dataset_name}.csv')
    train_seqs = train_df['seq']
    test_seqs  = test_df['seq']
    train_labels = train_df['label'].values
    test_labels  = test_df['label'].values

    # Encode labels
    le = LabelEncoder()
    y_train_enc = le.fit_transform(train_labels)
    y_test_enc  = le.transform(test_labels)
    y_train_oh  = to_categorical(y_train_enc)
    y_test_oh   = to_categorical(y_test_enc)

    combined_train_raw = []
    combined_test_raw = []

    # === Individual Embedding Models ===
    for model_key in embedding_sources:
        key_train = f"train_{dataset_name}_{model_key}_Embeddings.npy"
        key_test  = f"test_{dataset_name}_{model_key}_Embeddings.npy"
        X_train_raw = matrices[key_train].astype('float32')
        X_test_raw  = matrices[key_test].astype('float32')

        combined_train_raw.append(X_train_raw)
        combined_test_raw.append(X_test_raw)

        X_train = X_train_raw.reshape((X_train_raw.shape[0], X_train_raw.shape[1], 1))
        X_test  = X_test_raw.reshape((X_test_raw.shape[0],  X_test_raw.shape[1],  1))

        model = build_1d_cnn(input_shape=X_train.shape[1:])
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        print(f"\n--- Training {model_key} on {dataset_name} ---")
        model.fit(X_train, y_train_oh, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

        y_pred_probs = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred_probs, axis=1)
        y_test_classes = np.argmax(y_test_oh, axis=1)

        f1 = f1_score(y_test_classes, y_pred_classes)
        auroc = roc_auc_score(y_test_oh, y_pred_probs)
        prec, rec, _ = precision_recall_curve(y_test_oh.ravel(), y_pred_probs.ravel())
        auprc = auc(rec, prec)
        sens, spec = sensitivity_specificity(y_test_classes, y_pred_classes)
        acc = accuracy_score(y_test_classes, y_pred_classes)

        all_results.append([dataset_name, model_key, acc, f1, auroc, auprc, sens, spec])

    # === Combined Embedding Features ===
    print(f"\n--- Combined Features CNN on {dataset_name} ---")
    X_train_combined = np.concatenate(combined_train_raw, axis=1)
    X_test_combined  = np.concatenate(combined_test_raw, axis=1)

    X_train = X_train_combined.reshape((X_train_combined.shape[0], X_train_combined.shape[1], 1))
    X_test  = X_test_combined.reshape((X_test_combined.shape[0], X_test_combined.shape[1], 1))

    model = build_1d_cnn(input_shape=X_train.shape[1:])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train_oh, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

    y_pred_probs = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)
    y_test_classes = np.argmax(y_test_oh, axis=1)

    f1 = f1_score(y_test_classes, y_pred_classes)
    auroc = roc_auc_score(y_test_oh, y_pred_probs)
    prec, rec, _ = precision_recall_curve(y_test_oh.ravel(), y_pred_probs.ravel())
    auprc = auc(rec, prec)
    sens, spec = sensitivity_specificity(y_test_classes, y_pred_classes)
    acc = accuracy_score(y_test_classes, y_pred_classes)

    all_results.append([dataset_name, 'CombinedFeatures', acc, f1, auroc, auprc, sens, spec])

    # === Character-level CNN Baseline ===
    print(f"\n--- CharTokenizer Baseline on {dataset_name} ---")
    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts(train_seqs)
    train_seq_int = tokenizer.texts_to_sequences(train_seqs)
    test_seq_int  = tokenizer.texts_to_sequences(test_seqs)

    max_len = max(max(len(s) for s in train_seq_int), max(len(s) for s in test_seq_int))
    X_train = pad_sequences(train_seq_int, maxlen=max_len).reshape(-1, max_len, 1)
    X_test  = pad_sequences(test_seq_int,  maxlen=max_len).reshape(-1, max_len, 1)

    model = build_1d_cnn(input_shape=(max_len, 1))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train_enc, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    acc = accuracy_score(y_test_enc, y_pred_classes)
    f1 = f1_score(y_test_enc, y_pred_classes)
    auroc = roc_auc_score(y_test_enc, y_pred[:, 1])
    auprc = average_precision_score(y_test_enc, y_pred[:, 1])
    sens, spec = sensitivity_specificity(y_test_enc, y_pred_classes)

    print("CharTokenizer Results:")
    print(f"Accuracy: {acc}, F1: {f1}, AUROC: {auroc}, AUPRC: {auprc}, Sensitivity: {sens}, Specificity: {spec}")

    all_results.append([dataset_name, 'CharTokenizer', acc, f1, auroc, auprc, sens, spec])

# === Save Results ===
results_df = pd.DataFrame(
    all_results,
    columns=['Dataset', 'Feature_Set', 'Accuracy', 'F1_Score', 'AUROC', 'AUPRC', 'Sensitivity', 'Specificity']
)
results_df.to_csv(f'{base_dir}/Result3_R1 All_Dataset_Results.csv', index=False)
print("\nSaved results to: Result3_R1 All_Dataset_Results.csv")


Loading from folder: DNA_BERT2
  Reading file: test_demo_coding_vs_intergenomic_seqs_DB2_Embeddings.npy
  Reading file: test_human_enhancers_cohn_DB2_Embeddings.npy
  Reading file: test_human_enhancers_ensembl_DB2_Embeddings.npy
  Reading file: test_human_ensembl_regulatory_DB2_Embeddings.npy
  Reading file: test_human_nontata_promoters_DB2_Embeddings.npy
  Reading file: test_human_ocr_ensembl_DB2_Embeddings.npy
  Reading file: train_demo_coding_vs_intergenomic_seqs_DB2_Embeddings.npy
  Reading file: train_human_enhancers_cohn_DB2_Embeddings.npy
  Reading file: train_human_enhancers_ensembl_DB2_Embeddings.npy
  Reading file: train_human_ensembl_regulatory_DB2_Embeddings.npy
  Reading file: train_human_nontata_promoters_DB2_Embeddings.npy
  Reading file: train_human_ocr_ensembl_DB2_Embeddings.npy
Loading from folder: HyenaDNA
  Reading file: test_demo_coding_vs_intergenomic_seqs_HD_Embeddings.npy
  Reading file: test_human_enhancers_cohn_HD_Embeddings.npy
  Reading file: test_human_enha

/home/rxa615/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



--- Training DB2 on demo_coding_vs_intergenomic_seqs ---
Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 45s 24ms/step - accuracy: 0.8377 - loss: 0.3599 - val_accuracy: 0.8361 - val_loss: 0.3663
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 44s 23ms/step - accuracy: 0.8968 - loss: 0.2565 - val_accuracy: 0.8103 - val_loss: 0.4386
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 23ms/step - accuracy: 0.9046 - loss: 0.2396 - val_accuracy: 0.8346 - val_loss: 0.3867
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 23ms/step - accuracy: 0.9090 - loss: 0.2251 - val_accuracy: 0.8729 - val_loss: 0.3091
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 23ms/step - accuracy: 0.9122 - loss: 0.2206 - val_accuracy: 0.9073 - val_loss: 0.2500
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 23ms/step - accuracy: 0.9119 - loss: 0.2198 - val_accuracy: 0.8859 - val_loss: 0.2905
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 23ms/step - accuracy: 0.9123 - loss: 0.2169 - val_accuracy: 0.8063 - val_loss: 0.4624
Epoch 8/10
1875/18

/home/rxa615/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.7997 - loss: 0.4324 - val_accuracy: 0.7985 - val_loss: 0.4494
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.8491 - loss: 0.3480 - val_accuracy: 0.7735 - val_loss: 0.4755
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.8581 - loss: 0.3310 - val_accuracy: 0.7319 - val_loss: 0.5137
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.8591 - loss: 0.3288 - val_accuracy: 0.8099 - val_loss: 0.4513
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.8635 - loss: 0.3230 - val_accuracy: 0.7697 - val_loss: 0.5168
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.8646 - loss: 0.3179 - val_accuracy: 0.7844 - val_loss: 0.4931
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.8693 - loss: 0.3149 - val_accuracy: 0.7777 - val_loss: 0.4854
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.8645 - loss: 0.3159 - val

...

678/678 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.8131 - loss: 0.4500 - val_accuracy: 0.7452 - val_loss: 0.4367
Epoch 2/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 15s 22ms/step - accuracy: 0.8506 - loss: 0.3630 - val_accuracy: 0.7203 - val_loss: 0.5398
Epoch 3/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 15s 22ms/step - accuracy: 0.8585 - loss: 0.3447 - val_accuracy: 0.7428 - val_loss: 0.4750
Epoch 4/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 15s 22ms/step - accuracy: 0.8672 - loss: 0.3129 - val_accuracy: 0.7900 - val_loss: 0.3727
Epoch 5/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 15s 22ms/step - accuracy: 0.8763 - loss: 0.2897 - val_accuracy: 0.7399 - val_loss: 0.4696
Epoch 6/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 15s 22ms/step - accuracy: 0.8862 - loss: 0.2639 - val_accuracy: 0.7428 - val_loss: 0.4867
Epoch 7/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 15s 22ms/step - accuracy: 0.8924 - loss: 0.2497 - val_accuracy: 0.8229 - val_loss: 0.3423
Epoch 8/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 15s 22ms/step - accuracy: 0.9009 - loss: 0.2273 - val_accurac

/home/rxa615/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 110s 161ms/step - accuracy: 0.8150 - loss: 0.5053 - val_accuracy: 0.7430 - val_loss: 0.4704
Epoch 2/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 107s 158ms/step - accuracy: 0.8686 - loss: 0.3213 - val_accuracy: 0.7620 - val_loss: 0.4444
Epoch 3/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 106s 156ms/step - accuracy: 0.8773 - loss: 0.2991 - val_accuracy: 0.7694 - val_loss: 0.4315
Epoch 4/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 107s 157ms/step - accuracy: 0.8844 - loss: 0.2729 - val_accuracy: 0.8050 - val_loss: 0.3376
Epoch 5/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 106s 156ms/step - accuracy: 0.8946 - loss: 0.2499 - val_accuracy: 0.7625 - val_loss: 0.5609
Epoch 6/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 106s 157ms/step - accuracy: 0.9043 - loss: 0.2253 - val_accuracy: 0.8352 - val_loss: 0.3475
Epoch 7/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 106s 156ms/step - accuracy: 0.9128 - loss: 0.2051 - val_accuracy: 0.8225 - val_loss: 0.4826
Epoch 8/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 106s 157ms/step - accuracy: 0.9246 -

/home/rxa615/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


678/678 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.6748 - loss: 0.6212 - val_accuracy: 0.7762 - val_loss: 0.4086
Epoch 2/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8151 - loss: 0.4405 - val_accuracy: 0.7884 - val_loss: 0.3814
Epoch 3/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8304 - loss: 0.4025 - val_accuracy: 0.7640 - val_loss: 0.4365
Epoch 4/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8405 - loss: 0.3791 - val_accuracy: 0.7478 - val_loss: 0.4371
Epoch 5/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8536 - loss: 0.3531 - val_accuracy: 0.7653 - val_loss: 0.4282
Epoch 6/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8552 - loss: 0.3454 - val_accuracy: 0.7701 - val_loss: 0.4320
Epoch 7/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8646 - loss: 0.3202 - val_accuracy: 0.7474 - val_loss: 0.4841
Epoch 8/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8664 - loss: 0.3151 - val_accuracy: 0.7262 - val_

In [7]:
results_df

,Dataset,Feature_Set,Accuracy,F1_Score,AUROC,AUPRC,Sensitivity,Specificity
0,demo_coding_vs_intergenomic_seqs,DB2,0.903920,0.907537,0.970698,0.965869,0.943040,0.864800
1,demo_coding_vs_intergenomic_seqs,HD,0.847320,0.859291,0.940909,0.929279,0.932400,0.762240
2,demo_coding_vs_intergenomic_seqs,mistral,0.868840,0.876269,0.953685,0.945818,0.928880,0.808800
3,demo_coding_vs_intergenomic_seqs,NT,0.869040,0.875702,0.950481,0.948412,0.922640,0.815440
4,demo_coding_vs_intergenomic_seqs,GENALM,0.931880,0.932536,0.981178,0.980875,0.941600,0.922160
5,demo_coding_vs_intergenomic_seqs,GROVER,0.903800,0.904026,0.966193,0.966500,0.906160,0.901440
6,demo_coding_vs_intergenomic_seqs,CombinedFeatures,0.929080,0.930238,0.979953,0.979736,0.945680,0.912480
7,demo_coding_vs_intergenomic_seqs,CharTokenizer,0.840360,0.852004,0.930585,0.927388,0.919040,0.761680
8,human_enhancers_cohn,DB2,0.748705,0.735695,0.834464,0.828758,0.699482,0.797927
9,human_enhancers_cohn,HD,0.712435,0.698733,0.797455,0.793957,0.666955,0.757916
